Importando bibliotecas

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm 
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_curve, roc_auc_score, classification_report

Criação de dataset

In [ ]:
np.random.seed(42)
n = 200
idade = np.random.randint(18, 65, size=n)
renda = np.random.randint(2000, 10000, size=n)
p = 1 / (1 + np.exp(-(0.02*(idade - 35) + 0.0006*(renda - 5000))))
compra = np.random.binomial(1, p)

df = pd.DataFrame({'idade': idade, 'renda': renda, 'compra': compra})
display(df.head())

Separar variáveis dependentes, independentes e segmentos de treino e validação

In [ ]:
X = df[['idade', 'renda']]
y = df['compra']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

Treinar modelo

In [ ]:
model = LogisticRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:, 1]

Métricas principais

In [ ]:
print("Acurácia:", accuracy_score(y_test, y_pred))
print("Precisão:", precision_score(y_test, y_pred))
print("Recall (Sensibilidade):", recall_score(y_test, y_pred))
print("F1-score:", f1_score(y_test, y_pred))
print("AUC-ROC:", roc_auc_score(y_test, y_prob))

Matriz de confusão

In [ ]:
cm = confusion_matrix(y_test, y_pred)
print("Matriz de confusão:\n", cm)

Relatório completo

In [ ]:
print(classification_report(y_test, y_pred))

Curva ROC e AUC

In [ ]:
fpr, tpr, _ = roc_curve(y_test, y_prob)
plt.plot(fpr, tpr, label=f"AUC = {roc_auc_score(y_test, y_prob):.2f}")
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel("Taxa de falsos positivos (1 - Especificidade)")
plt.ylabel("Taxa de verdadeiros positivos (Sensibilidade)")
plt.title("Curva ROC")
plt.legend()
plt.show()

Análise dos coeficientes e confiabilidade

In [ ]:
X2 = sm.add_constant(X)  # adiciona intercepto
logit_model = sm.Logit(y, X2)
result = logit_model.fit()

print(result.summary())

Diagnóstico e confiabilidade do modelo - Pseudo-R² (McFadden)

In [ ]:
print("Pseudo R² (McFadden):", 1 - result.llf / result.llnull)

Verificação de Multicolinearidade

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = pd.DataFrame()
vif["Variável"] = X.columns
vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
print(vif)